In [ ]:

import os, json
from os import listdir
from os.path import isfile, join
from tqdm import tqdm
import plotly.graph_objects as go

base_dir = rf"C:\NotOneDrive\Data\for_hci"
profile_dir = join(base_dir, r"BDC1_che_temp_profile")
mode = 'in_drillstring' # in_string
file_names = [f for f in listdir(profile_dir) if isfile(join(profile_dir, f))]
file_names.sort()
# print(file_names[0], file_names[1])
file_names = file_names[::50]
profiles = []
for fn in tqdm(file_names):
    fp = join(profile_dir, fn)
    with open(fp, 'r') as f:
        content = json.load(f)

    if len(content) == 0:
        continue

    profiles.append(content)

In [ ]:

indexes = list(range(len(profiles)))
surface_points = [x[0]['Value'] for x in profiles]
# print(surface_points)

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        mode = 'markers',
        marker = dict(
            size = 3
        ),
        x=indexes, 
        y=surface_points, 
        name="difference ecd at bit",
        yaxis='y'),
)

fig.update_layout(
    title = fr'Mud {mode} temperature at md = 0.0',
    )
fig.update_yaxes(title = 'temperature (K)')
fig.update_xaxes(title = 'time index')
# fig.add_vline(x=packoff_time_key, line_width=3, line_dash="dash", line_color="green", name = 'Stuckpipe detected')
# fig.write_html(os.path.join(r"C:\NotOneDrive\Data\for_hci", fr"{well_name}_TransientHydraulics_ECD_{time_stamp_str}.html"))
save_path = os.path.join(base_dir, f"mud_{mode}_temp_at_surface.html")
fig.write_html(save_path)
fig.show()

In [ ]:

formation_temp_points = {
    'index': [],
    'depth': [],
    'temp': [],
}

i = 0
for c in profiles:
    for _c in c:
        formation_temp_points['depth'].append(_c['Key'])
        formation_temp_points['index'].append(i)
        formation_temp_points['temp'].append(_c['Value'])
        i += 1


fig = go.Figure()
fig.add_trace(
    go.Scatter(
        mode="markers",
        x=formation_temp_points['index'], 
        y=formation_temp_points['depth'], 
        marker_color = formation_temp_points['temp'],
        marker_colorscale='Plasma',
        marker_showscale=True,
    ),
)

fig.update_layout(title = f'Mud {mode} temperature profile computed by RTCHE',
                    title_font=dict(size=25,
                                color='blue',
                                family='Arial')
                )

fig.update_yaxes(autorange = 'reversed', title = 'depth')
fig.update_xaxes(title = 'time index')
fig.show()
save_path = os.path.join(base_dir, f"mud_{mode}_temp_profile.html")
fig.write_html(save_path)